In [3]:
!pip install torch torchvision


  Using cached torch-2.1.2-cp38-cp38-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.16.2-cp38-cp38-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached fsspec-2023.12.2-py3-none-any.whl.metadata (6.8 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached triton-2.1.0-0-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.3 kB)
Using cached torch-2.1.2-cp38-cp38-manylinux1_x86_64.whl (670.2 MB)
Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
Using cached triton-2.1.0-0-cp38-cp38-manyli

In [ ]:
%pip install tensorflow

In [60]:
import subprocess
import json
import base64

# Replace the server URL with your server address
server = "https://127.0.0.1:8000"

def create_custom_model():
    # Define your custom CNN model here
    # Example: a simple CNN with two convolutional layers and one dense layer
    model_definition = {
        "layers": [
            {"type": "conv2d", "filters": 64, "kernel_size": 3, "activation": "relu", "input_shape": [28, 28, 1]},
            {"type": "maxpooling2d", "pool_size": [2, 2]},
            {"type": "conv2d", "filters": 32, "kernel_size": 3, "activation": "relu"},
            {"type": "maxpooling2d", "pool_size": [2, 2]},
            {"type": "flatten"},
            {"type": "dense", "units": 10, "activation": "softmax"}
        ]
    }
    return model_definition

def serialize_custom_model(model_definition):
    # Serialize the model definition to JSON
    return json.dumps(model_definition)

def encode_to_base64(data):
    # Encode data to Base64
    return base64.b64encode(data.encode()).decode()

def upload_initial_custom_model(json_payload, user_cert, user_key):
    # Upload the initial custom model to the server
    payload = {
       "global_model":{
            "model_name": "CustomModel",
        "model_data": encode_to_base64(json_payload),
       }
    }

    response = subprocess.run([
        "curl",
        "-X", "POST",
        f"{server}/app/model",
        "--cacert", "./workspace/sandbox_common/service_cert.pem",
        "--cert", user_cert,
        "--key", user_key,
        "-H", "Content-Type: application/json",
        "--data-raw", json.dumps(payload)
    ], capture_output=True, text=True)

    print("Response from server:", response.stdout)

    if response.returncode == 0:
        print("Initial custom model uploaded successfully.")
        return True
    else:
        print(f"Failed to upload initial custom model. Error: {response.stderr}")
        return False

# Initialize and upload initial custom model with JSON data as payload
json_payload = serialize_custom_model(create_custom_model())
if not upload_initial_custom_model(json_payload, "./workspace/sandbox_common/user0_cert.pem", "./workspace/sandbox_common/user0_privk.pem"):
    raise Exception("Failed to upload the initial custom model. Stopping the process.")


Response from server: {"message":"Model uploaded successfully","model_id":3,"model_name":"CustomModel"}
Initial custom model uploaded successfully.


In [77]:
import subprocess
import json
import base64

# Replace the server URL with your server address
server = "https://127.0.0.1:8000"

def download_custom_model(model_id, user_cert, user_key):
    # Specify the model ID to download
    download_payload = {
        "model_id": model_id
    }

    response = subprocess.run([
        "curl",
        "-X", "GET",
        f"{server}/app/model?model_id={model_id}",
        "--cacert", "./workspace/sandbox_common/service_cert.pem",
        "--cert", user_cert,
        "--key", user_key,
        "-H", "Content-Type: application/json",
 
    ], capture_output=True, text=True)

    print("Response from server:", response.stdout)

    if response.returncode == 0:
        print("Model downloaded successfully.")
        # Decode and return the model data
        print(response.stdout)
        model_data = base64.b64decode(json.loads(response.stdout)["model_details"]).decode()
        return model_data
    else:
        print(f"Failed to download model. Error: {response.stderr}")
        return None

# Replace MODEL_ID with the actual model ID you want to download
MODEL_ID = 3  # Replace with the actual model ID
downloaded_model = download_custom_model(MODEL_ID, "./workspace/sandbox_common/user0_cert.pem", "./workspace/sandbox_common/user0_privk.pem")

if downloaded_model:
    # Now 'downloaded_model' contains the downloaded model data, you can use it as needed
    print("Downloaded model data:")
    print(downloaded_model)


Response from server: {"error":{"code":"ResourceNotFound","message":"Cannot find model for id \"3\"."}}
Model downloaded successfully.
{"error":{"code":"ResourceNotFound","message":"Cannot find model for id \"3\"."}}


KeyError: 'model_details'

In [83]:
import requests
from requests.exceptions import ConnectionError
import time

max_retries = 3
retry_delay = 2  # seconds

for attempt in range(1, max_retries + 1):
    try:
        headers = {
            "Content-Type": "application/json",
        }
        print(f"Attempt {attempt} to connect to the server...")
        response = requests.get(f"{server}/app/model?model_id={MODEL_ID}", headers=headers, verify="./workspace/sandbox_common/service_cert.pem", cert=("./workspace/sandbox_common/user0_cert.pem", "./workspace/sandbox_common/user0_privk.pem"))
        print("Response from server:", response.text)
        response.raise_for_status()
        
        # Process the response here
        break  # Success, exit the loop
    except ConnectionError as e:
        print("connect error",e)
        print(f"Attempt {attempt} failed. Retrying in {retry_delay} seconds.")
        time.sleep(retry_delay)
else:
    # All attempts failed
    print("Maximum number of retries reached. Could not establish a connection.")


Attempt 1 to connect to the server...
connect error ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Attempt 1 failed. Retrying in 2 seconds.
Attempt 2 to connect to the server...
connect error ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Attempt 2 failed. Retrying in 2 seconds.
Attempt 3 to connect to the server...
connect error ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Attempt 3 failed. Retrying in 2 seconds.
Maximum number of retries reached. Could not establish a connection.
